In [1]:
import json
from typing import Any, Dict

In [2]:
REPO_SETS = [
    'eval/resources/python_repos_5k-1k.json',
    'eval/resources/python_repos_10k-5k.json',
    'eval/resources/python_repos_20k-10k.json',
    'eval/resources/python_repos_20k+.json'
]
DOCKERFILE_PATH = "resources/working_dockerfiles"
def load_repos():
    repos = {repo_set: json.load(open(repo_set, 'r')) for repo_set in REPO_SETS}
    return repos

In [3]:
repos = load_repos()

In [4]:
def calc_visbility(repo: Dict[str, Any]):
    # Calculate the "visibility" of a repo's documentation based on the following heursitic
    # 1 / (num directories to navigate + num files to find)
    dirs = [relevant.split('/') for relevant in repo['relevant_docs']]
    paths = [['/'.join(repo_dirs[:i+1]) for i in range(len(repo_dirs))] for repo_dirs in dirs]
    visibility = set([d for directories in paths for d in directories])
    return 1 / len(visibility) if len(visibility) > 0 else 0

In [6]:
for repo in repos[REPO_SETS[-1]]:
    print(repo['url'])
    print(" - " + ', '.join(repo['relevant_docs']))
    visivility = calc_visbility(repo)
    repo['visivility'] = visivility
    print(" - " + str(visivility))


https://github.com/tiangolo/fastapi.git
 - docs/en/docs/contributing.md, docs/en/docs/tutorial/testing.md
 - 0.16666666666666666
https://github.com/psf/black.git
 - docs/contributing/the_basics.md
 - 0.3333333333333333
https://github.com/tqdm/tqdm.git
 - 
 - 0
https://github.com/Textualize/rich.git
 - CONTRIBUTING.md
 - 1.0
https://github.com/OpenInterpreter/open-interpreter.git
 - docs/CONTRIBUTING.md
 - 0.5
https://github.com/home-assistant/core.git
 - 
 - 0
https://github.com/sherlock-project/sherlock.git
 - 
 - 0
https://github.com/explosion/spaCy.git
 - README.md
 - 1.0
https://github.com/soimort/you-get.git
 - README.md
 - 1.0
https://github.com/Textualize/textual.git
 - CONTRIBUTING.md
 - 1.0


In [5]:
import os
from difflib import get_close_matches
from pprint import pprint
from tqdm import tqdm
from doc_test.agent.functions import _get_file_contents, get_api_url

def get_dockerfile(repo, set_name):
    name = repo['url'].split('/')[-1][:-4]
    set_name = set_name.split('_')[-1][:-5]
    dockerfile_path = os.path.join(DOCKERFILE_PATH, set_name, name.lower()+'.dockerfile')
    if not os.path.exists(dockerfile_path):
        print(f"path {dockerfile_path} does not exist")
        return -2, []
    dockerfile = open(dockerfile_path, 'r').readlines()

    return dockerfile

def calc_informativity(repo: Dict[str, Any], set_name: str):
    # Calculate the "informativity" of a repo's documentation based on the following heuristic
    # (Num lines in model dockerfile that appear in documentation) / (Num lines in model dockerfile total)

    # step 1: load dockerfile
    dockerfile = get_dockerfile(repo, set_name)

    dockerfile = [
        ' '.join(line.split(' ')[1:]).strip()
        for line in dockerfile
        if not (
            line.startswith('COPY') or line.startswith('WORKDIR') or line.strip() == ''
        )
    ]

    # step 2: load documents
    if len(repo['relevant_docs']) == 0:
        return -1, []
    api_url = get_api_url(repo['url'])
    documents = [_get_file_contents(api_url, path, repo['ref']).split('\n') for path in repo['relevant_docs']]

    # step 3: find dockerfile lines that occur in documentation
    matched_lines = {}
    for doc, doc_name in zip(documents, repo['relevant_docs']):
        for line in doc:
            matches = get_close_matches(line, dockerfile, cutoff=0.8)
            if len(matches) > 0:
                if matches[0] not in matched_lines:
                    matched_lines[matches[0]] = []
                matched_lines[matches[0]].append((line, doc_name))
            else:
                for df_line in dockerfile:
                    if df_line in line:
                        if df_line not in matched_lines:
                            matched_lines[df_line] = []
                        matched_lines[df_line].append((line, doc_name))

    return (len(matched_lines) / len(dockerfile), matched_lines)

In [8]:
# CALCULATE BOTH HEURISTICS FOR ALL REPOS
for repo_set in REPO_SETS:
    for repo in repos[repo_set]:
        print(repo['url'])
        print(" - " + ', '.join(repo['relevant_docs']))

        info = calc_informativity(repo, repo_set)
        repo['info'] = info[0]
        print(" - informativity = " + str(info[0]))

        visivility = calc_visbility(repo)
        repo['visivility'] = visivility
        print(" - visibility = " + str(visivility))

https://github.com/mandarons/icloud-drive-docker.git
 - 
 - informativity = -1
 - visibility = 0
https://github.com/typeddjango/django-stubs.git
 - CONTRIBUTING.md
 - informativity = 0.3333333333333333
 - visibility = 1.0
https://github.com/PennyLaneAI/pennylane.git
 - .github/CONTRIBUTING.md, doc/development/guide/installation.rst, doc/development/guide/tests.rst
 - informativity = 0.4
 - visibility = 0.14285714285714285
https://github.com/CVHub520/X-AnyLabeling.git
 - docs/en/get_started.md
 - informativity = 0.4
 - visibility = 0.3333333333333333
https://github.com/open-compass/opencompass.git
 - README.md, docs/en/get_started/installation.md
 - informativity = 0.4
 - visibility = 0.2
https://github.com/SciPhi-AI/R2R.git
 - 
 - informativity = -1
 - visibility = 0
https://github.com/VainF/Torch-Pruning.git
 - README.md
 - informativity = 0.0
 - visibility = 1.0
https://github.com/pypi/warehouse.git
 - docs/dev/development/getting-started.rst
 - informativity = 0.3333333333333333
 - 

In [9]:
# WRITE HEURISTICS  
for repo_set in REPO_SETS:
    with open(repo_set, 'w') as f:
        json.dump(repos[repo_set], f)

In [10]:
from vm_control import VMController, test_dockerfile
from doc_test.utils import notify
# TEST DOCKERFILES
results = {}
i = 0
for repo_set in REPO_SETS:
    print(repo_set)
    for repo in repos[repo_set]:
        name = repo['url'].split('/')[-1][:-4]
        vmc = VMController(f"working_logs/{name}.log")
        results[name] = test_dockerfile(
            repo['url'],
            "\n".join(get_dockerfile(repo, repo_set)),
            name,
            vmc,
            repo['ref']
        )
        vmc.clear_cache()
        i += 1
        notify(f" - ({i}/40) {name}: {results[name]}")

eval/resources/python_repos_5k-1k.json
# Doesnt work with python 3.8

FROM python:3.10



COPY . /app/



WORKDIR /app



RUN pip install -r requirements.txt

RUN pip install -r requirements-test.txt



# Commands to create required directories, taken from repo's workflows

RUN mkdir /config /icloud

RUN chown $(id -u) /config /icloud



RUN pytest

attempting to build using dockerfile, logs written to working_logs/icloud-drive-docker.log.
/tmp/tmp.KUJTRCcgAp/icloud-drive-docker


At least 1 test passed.
Docker build completed successfully on virtual machine.


Error response from daemon: No such image: temp_image:latest


Deleted build cache objects:
jcra2yibdmcycfqdbj2zem78w
gy6y9tfm1250nqqw63auugcwy
uh21sfbtxvui2wi89ikbzzg7e
s8hq03jgxpgnmiz38v3bvc327
ioeqjv88rz3qzxjijrc4k5rpd
4otzfnzo62ixw7z3l9oq3c9x6
loays0oed0tn0f73capkqa1jy
kf59ptmt34jlsk8qvnosm5bz8
ikm13gcp96q73gwvm9q9xmiw5
zmxvn85z89ol4jmx6qzxdb8s5
l1z7r0bq2ai878mvyic2ngf2p
h9bbdy2flez0obf5uddc9wnpb
rxa05tsh3rgkwsl8j7aw9acbp
ywn17jhb2u6t6rx5rlblrvfkv
128ib7h8eljkwqx11avfi9mun
qogq1h7ta064ov6w2dj3e2hs1

Total reclaimed space: 216.6MB
 - (1/40) icloud-drive-docker: True
FROM python:3.10



COPY . /app/



WORKDIR /app



RUN pip install -r requirements.txt



RUN pytest

attempting to build using dockerfile, logs written to working_logs/django-stubs.log.
/tmp/tmp.Uh0qegIaSA/django-stubs


At least 1 test passed.
Docker build completed successfully on virtual machine.


...

At least 1 test passed.
Docker build completed successfully on virtual machine.
Untagged: temp_image:latest
Deleted: sha256:6da1f8160d5b5f9fa5d02d5ab7e9c465b37cd1bb87b7457478d67994c0e349c2
Deleted build cache objects:
01abrcp9z8fwi85jjsqcrhgtc
v2o134bwkbugegvc6ooqlh5ia
mp8mt3bw721ueqszzmqa81v7g
bhp4mugyna9ydnkg4nvvn7ouq
gl9x8piwu6s33blr6gx06jx3i
srfr76yjjio8el9etweghy8qc
yq35fdzi1wv94ha339z7vgggi
nq2jzsscxvv7qj8k6h63wnigz
yfs84h8g561xriyl1isuyrw8y
n8aqyved1w3147jnrywyay2ej
oljl6sdb6rwxl1oyd8vho0040
7av7fxbp6zbqz4hpwyte1fhb7
t3zhe2jybx9nd9mnxi67u5ril
y1iwreswalrwdz00ei207ztgn
l92el7sxoir6ww88h2rf1hq8y
ihzrkjn2ths256ta5ffw2sqeo

Total reclaimed space: 1.153GB
 - (38/40) spaCy: True
FROM python:3.9



COPY . /app/



WORKDIR /app



RUN pip install -r requirements.txt

RUN pip install -e .



RUN python -m unittest discover

attempting to build using dockerfile, logs written to working_logs/you-get.log.
/tmp/tmp.poGuaIiq6W/you-get


At least 1 test passed.
Docker build completed successfully on virtual machine.
Untagged: temp_image:latest
Deleted: sha256:e706de82c76e908cf6fd11cec3db97f838c624dbaa4790c6bb4d073694e03c22
Deleted build cache objects:
p2ra6r9wddjtlmhsxpgxqp6y1
w6flpqzbjh3ho5pdmv7hw2ivq
7ulw8jxo12fhr2zkahtife2tq
uogtf0usxltqhzj6rkpr30b4w
m9eswey11373mrxuq5liqzt2e
w66n50i3onvnmpx2dyhnqhh3i
1iiwhx34ofjdt9z0lia9g9zz4
pktf5op9ib6kiyi8q4x3tynt1
z8s5bjeu2e12u0cpsjt5umm0e
msepjj2m8j57mlvp3tjktlke3
fhyka6zfn1esyf95x9jrb1fl0
t3f2u3gf9j3scnr522fidsgss
pemyebzdsyni1suytlx7cjn0k
xdros3h1xa1whsmbm2f3emfke
ogv570agrla3t5btsin6xwn0n

Total reclaimed space: 30.27MB
 - (39/40) you-get: True
FROM python:3.8



WORKDIR /app



COPY . /app/



RUN pip install poetry



RUN poetry install



RUN poetry run pip install pytest



RUN poetry run pytest


attempting to build using dockerfile, logs written to working_logs/textual.log.
/tmp/tmp.10ddVTKYh6/textual


At least 1 test passed.
Docker build completed successfully on virtual machine.


Error response from daemon: No such image: temp_image:latest


Deleted build cache objects:
y33ljgjnkjqd95gjd2xvaaj7x
6jelabevopu7y4upbr8221drr
wz7xplirtu13jyrndt7bw6772
do96nq47qmdatbj8f88v2qwdg
ewhxlkjyg2sl7eyl9zwoivn8m
lqg9xy822bwn58bj0xugwfmxl
egd0mum05wvshoqz16tn78wn5
nnu6ve2mw3orz1y207lqbylpz
whtatz38ajabrp6h3u55uylpe
u6roan50trjbjqfxt6k5ne048
6imbz3e0rno0hpy06vm7gvnrf
7g4k2dq9uy7hh5tk3147z8u3i
pbvyb76crsjkngq1749p2pjse
l89qa7lg5en2m4u3qgmlb72ep
u4nr72lhppavudz5lk486ej1n

Total reclaimed space: 1.241GB
 - (40/40) textual: True


In [11]:
results

{'icloud-drive-docker': True,
 'django-stubs': True,
 'pennylane': True,
 'X-AnyLabeling': True,
 'opencompass': False,
 'R2R': False,
 'Torch-Pruning': False,
 'warehouse': False,
 'sabnzbd': True,
 'dlt': True,
 'camel': True,
 'boto3': False,
 'cloud-custodian': False,
 'aim': False,
 'speechbrain': True,
 'nonebot2': True,
 'moto': True,
 'instructor': False,
 'numba': True,
 'pymc': True,
 'yfinance': False,
 'beets': True,
 'starlette': True,
 'datasets': False,
 'mypy': True,
 'sympy': False,
 'ydata-profiling': True,
 'spotify-downloader': True,
 'qlib': True,
 'scapy': True,
 'fastapi': True,
 'black': True,
 'tqdm': True,
 'rich': True,
 'open-interpreter': True,
 'core': False,
 'sherlock': True,
 'spaCy': True,
 'you-get': True,
 'textual': True}

In [6]:
from vm_control import VMController, test_dockerfile
from doc_test.utils import notify

def get_repo(name):
    return [(r, repo_set) for repo_set in REPO_SETS for r in repos[repo_set] if name.lower() in r['url'].lower()]

def test_repo(repo, repo_set):
    name = repo['url'].split('/')[-1][:-4]
    vmc = VMController(f"working_logs/{name}.log")
    result = test_dockerfile(
        repo['url'],
        "\n".join(get_dockerfile(repo, repo_set)),
        name,
        vmc,
        repo['ref']
    )
    vmc.clear_cache()
    return result

In [7]:
get_repo("scvi")[0]

({'url': 'https://github.com/scverse/scvi-tools.git',
  'categories': [4],
  'test_type': 'pytest',
  'relevant_docs': ['docs/developer/code.md'],
  'tags': ['install-self', 'pytest', 'pytest-extra'],
  'ref': 'e8f9b36'},
 'eval/resources/python_repos_5k-1k.json')

In [12]:
test_repo(*get_repo("scvi")[0])

FROM python:3.10

WORKDIR /app

COPY . /app/



RUN pip install -e ".[dev]"



RUN python -m pytest -x

attempting to build using dockerfile, logs written to working_logs/scvi-tools.log.
/tmp/tmp.CkSCrHVwDa/scvi-tools


At least 1 test passed.
Docker build completed successfully on virtual machine.


Error response from daemon: No such image: temp_image:latest


Deleted build cache objects:
iassgcruzgc3tr9j8uxen3c1s
bxnyo2fy2b42p2x86gqqxxoak
jibxs9u3elymjz9325tsx5rfw
wkrbtczvom6l5fl6vfnwy3062
azozrnz2699wlkwmqpxet1yj9
lk3hjhsg9bh57yswo8yj47wdf
rwxt085x2mxz21ffyx0fkf1yg
w5vy3n6likwi5r30edt94p8fq
pkn4o6w93y4rymxovivu4m4r6
nzgqp30qca5c486uny9ydxuh5
ga5b4uixh96ibxkxscvpphof0
blklpj2trz4fukgsw9t51asu0
gui7hekjk7jjr9j0nbu1ymnpb
plgwn0lios7kgin2a021oy709
fq6hnrhruuud3eltxobr0qybq
a6qyu3d8c6flryi575zctcphd

Total reclaimed space: 22.9GB


True

In [21]:
notify("come back!")

come back!


In [12]:
repeat = [
    'r2r',
    'torch-pruning',
    'boto3',
    'cloud-custodian',
    'aim',
    'instructor',
    'yfinance',
    'datasets',
    'sympy',
    'core'
    ]

In [17]:
for i, r in enumerate(repeat):
    success = test_repo(*get_repo(r)[0])
    notify(f"({i}/{len(repeat)}) {r}: {success}")

FROM python:3.10



COPY . /app/



WORKDIR /app



RUN pip install -e .



RUN python -m unittest discover

attempting to build using dockerfile, logs written to working_logs/opencompass.log.
/tmp/tmp.62i7JxsKSP/opencompass


At least 1 test passed.
Docker build completed successfully on virtual machine.
Untagged: temp_image:latest
Deleted: sha256:6f6a48576883e124fcf1b6d8248a045959c99e32fb54022c30f337fceb95439e
Deleted build cache objects:
vfo1myje4kkm1ioo4sh8xvxn9
zrcw7cdv3ibetgcq0j96nx0v7
ianbb455cmioxvex93ex6ypu6
1snzd7lbvzd16yrvw05getlzh
m3j4xe2h7n9x2qq4c68u0a0v0
sdy4nab14jd7zaeeasteil4ie
9v8erlkohv21mkns95i63tpjd
mf8fakufezsauc9qspu4l4fvc
qr0zfd6xmzt3nrul5lo0x84wb
r0f9705td7i63xwy8truwglvi
yke383yzije6hnbq0v54832gc
afvvlxd21zneach04tqdisvbz
pusm9qoxy9b73zp7qfpzh57jg
nj75dy0710nd7q5hgwqlrf6vl

Total reclaimed space: 9.247GB
(0/12) opencompass: True
FROM python:3.11

WORKDIR /app

COPY . /app/

RUN pip install poetry

RUN poetry install

RUN poetry run pytest


attempting to build using dockerfile, logs written to working_logs/R2R.log.
/tmp/tmp.wLJJKS70R3/R2R


Error response from daemon: No such image: temp_image:latest


Deleted build cache objects:
1q3rnz8iljmxngv1z2nieadru
ufm2ychjoakvqk3z1dj7vhysn
yt3yo5d7butcc9h3c4hdkny88
k1u32mzafpuwm3bcphubs1jjy
bm8xpqrb7aufl6tn6fylrps01
c884nju7wnitb3nzgjpv2ekjy
8o13nohj4w0k9m9wdiamhg7lb
u8ylnjwqabkep1sbftz31gt4l
n9f7myt6dd9ip2scgbkk4lw22
nar5d4hvzsgl4tqf87kn4qt9d
hy2b5twi5ze41q35m1ze1kghb
mz4pix5yhbaoh0tgg43dzamb3
rjrh3r6nngztkagx1erids3os

Total reclaimed space: 209.8MB
(1/12) r2r: False
FROM python:3.10

WORKDIR /app

COPY . /app/



RUN pip install -r requirements.txt

RUN pip install torchvision

RUN pip install pytest



RUN pytest

attempting to build using dockerfile, logs written to working_logs/Torch-Pruning.log.
/tmp/tmp.eHF0tkpwhB/Torch-Pruning


Error response from daemon: No such image: temp_image:latest


...

INTERRUPTING


Error response from daemon: No such image: temp_image:latest


Total reclaimed space: 0B
(10/12) sympy: False
FROM python:3.12



COPY . /app/



WORKDIR /app



RUN pip install -r requirements_all.txt

RUN pip install -r requirements_test_all.txt



RUN python -m unittest discover

attempting to build using dockerfile, logs written to working_logs/core.log.
/tmp/tmp.jHEGzQ6SO9/core


Error response from daemon: No such image: temp_image:latest


Deleted build cache objects:
w23rbu8i9j9931maodgopm4lb
xvxpj4rz1zd9dt7teqxsuut11
skf6lpnzp5w77szvqcjdnicpx
lc97xm45p4xy07exzvl9pyb3u
mdpi3oyfg150q6jczchq74yn4
23ep90s604evqt2v7uy731zoy
f927f7t35q401t1dpb6kzzkep
rabrkerayla8x95uuy21l25xh
l6qgwc4f1vy79g3nkayi5x6hp
uz95q3wbxpgvqaopeajhdgbw0
k9rguazt3pqenm6bssur1v3db
lexaozqxz5y7w714i8dxbpz8r
16br5uvi1ix2jqg2lr5338rzc
xxjxbtaeynm9l65c27lffytzs
yixgax0oyr4tvmavdxaw7o5um
jvslqbg6hzhf5f6b6owja8ifw
gpjfiynwu0a7vwf111ph8h45z
hesecll5o3ivs09nwhpo61iu3
pjigdjjcjg1c0gdo1he552ryt
2vfbes6c30g9448spirtoryfn
xu7sq7pa4vxc197ubck18xeya
xa7cwx49wjhs1x8tqoq5e8kl0
n3tcqfqy6suhbai8zgxaysmxo

Total reclaimed space: 4.283GB
(11/12) core: False
